In [ ]:
import numpy as np 
import pandas as pd 
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd
from skimage.transform import resize
import tensorflow as tf
from keras.callbacks import ModelCheckpoint, EarlyStopping ,ReduceLROnPlateau
from keras.layers.merge import concatenate
from keras import backend as K
from keras.layers.core import Lambda, RepeatVector, Reshape
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout 
import os
from keras.applications.vgg16 import preprocess_input

**Assigning the data Paths for the test, train , and Val sets.**

In [ ]:
train_images_path = "../input/assi4-dataset/resized_training/resized_training" 
train_mask_path = "../input/assi4-dataset/resized_training_mask/resized_training_mask"
train_metadata_path = "../input/assi4-dataset/ISIC-2017_Training_Data_metadata.csv"

val_images_path = "../input/assi4-dataset/resized_val/resized_val"
val_mask_path = "../input/assi4-dataset/resized_val_mask/resized_val_mask"
val_metadata_path = "../input/assi4-dataset/ISIC-2017_Validation_Data_metadata.csv"

test_images_path = "../input/assi4-dataset/resized_test/resized_test"
test_mask_path = "../input/assi4-dataset/resized_test_mask/resized_test_mask"
test_metadata_path = "../input/assi4-dataset/ISIC-2017_Test_v2_Data_metadata.csv"

**Auxillary Function for Reading the Meta Data**

In [ ]:
def read_metadata(path):
    metadata_Dataframe = pd.read_csv(path)
    metadata_Dataframe.head()
    return metadata_Dataframe

**Reading and Preparing the Images**

In [ ]:
def load_images_from_folder(path,ids):
    images = []
    for imageId in ids:
        images.append( cv2.resize(cv2.imread(os.path.join(path,imageId+'.jpg')), (256,256), interpolation = cv2.INTER_AREA) )
    print('images load done')
    return np.array(images)

**Reading and Preparing the Mask Images**

In [ ]:
def load_masks_from_folder(path,ids):
    images = []
    for imageId in ids:
        img = cv2.imread(os.path.join(path,imageId+'.png'))
        img = tf.image.resize(img, (256,256), method='nearest')
        img = tf.image.rgb_to_grayscale(img)/255
        images.append(img)
    print('images load done')
    return np.array(images)

In [ ]:
train_metadata = read_metadata(train_metadata_path)
val_metadata = read_metadata(val_metadata_path)
test_metadata = read_metadata(test_metadata_path)
train_images_ids = train_metadata['image_id']
val_images_ids = val_metadata['image_id']
test_images_ids = test_metadata['image_id']


train_images = load_images_from_folder(train_images_path,train_images_ids)
val_images = load_images_from_folder(val_images_path,val_images_ids)

test_images = load_images_from_folder(test_images_path,test_images_ids)

In [ ]:
train_masks = load_masks_from_folder(train_mask_path,train_images_ids)
val_masks = load_masks_from_folder(val_mask_path,val_images_ids)

test_masks = load_masks_from_folder(test_mask_path,test_images_ids)

**Checking the reading process and presenting Sample Images**

In [ ]:
plt.figure()
plt.title( "Train img ")
plt.imshow(train_images[0])
plt.figure()
plt.title( "Ground truth")
plt.imshow(train_masks[0])

plt.figure()
plt.title( "Val img")
plt.imshow(val_images[0])
plt.figure()
plt.title( "Ground truth")
plt.imshow(val_masks[0])

plt.figure()
plt.title( "Test img")
plt.imshow(test_images[0])
plt.figure()
plt.title( "Ground truth")
plt.imshow(test_masks[0])

In [ ]:
train_images.shape

**The CallBack Functions Used during the Training Process**

In [ ]:
callbacks = [
    EarlyStopping(patience=7, verbose=1), #verbose -> update msg
    ModelCheckpoint('model-v1.h5', verbose=0, save_best_only=True)
]



**The Jaccard IoU Loss Function used in our Training Process**

In [ ]:
from keras import backend as K
def jacard_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (intersection + 1.0) / (K.sum(y_true_f) + K.sum(y_pred_f) - intersection + 1.0)

def jacard_coef_loss(y_true, y_pred):
    return -jacard_coef(y_true, y_pred)  # -1 ultiplied as we want to minimize this value as loss function

# FCN-base-VGG16

In [ ]:
inputImg = tf.keras.layers.Input((256,256,3) , name='image')  #shape of each img = 256*256*3
#inputImg = Lambda(lambda x: x / 255)(inputImg)  #normalizing the input
c1 = tf.keras.layers.Conv2D(64, (3, 3),  activation='relu' , padding="same") (inputImg)
c1 = tf.keras.layers.Conv2D(64, (3, 3),  activation='relu' , padding="same") (c1)
p1 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2)) (c1)
#-------------------------------------------------------------------------------------
c2 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu' , padding="same") (p1)
c2 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu' , padding="same") (c2)
p2 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2)) (c2)
#-------------------------------------------------------------------------------------
c3 = tf.keras.layers.Conv2D(256, (3, 3),  activation='relu' , padding="same") (p2)
c3 = tf.keras.layers.Conv2D(256, (3, 3),  activation='relu' , padding="same") (c3)
c3 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu' , padding="same") (c3)
p3 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2)) (c3)
#-------------------------------------------------------------------------------------
c4 = tf.keras.layers.Conv2D(512, (3, 3),  activation='relu' , padding="same") (p3)
c4 = tf.keras.layers.Conv2D(512, (3, 3),  activation='relu' , padding="same") (c4)
c4 = tf.keras.layers.Conv2D(512, (3, 3),  activation='relu' , padding="same") (c4)
p4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2)) (c4)
#-------------------------------------------------------------------------------------
c5 = tf.keras.layers.Conv2D(512, (3, 3), activation='relu' , padding="same") (p4)
c5 = tf.keras.layers.Conv2D(512, (3, 3), activation='relu' , padding="same") (c5)
c5 = tf.keras.layers.Conv2D(512, (3, 3), activation='relu' , padding="same") (c5)
p5 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2)) (c5)
#-------------------------------------------------------------------------------------

Added_c6 = tf.keras.layers.Conv2D(4096, (1, 1),  kernel_initializer='he_uniform',activation='relu' , padding="same") (p5)
Added_c6 = tf.keras.layers.Conv2D(1, (1, 1), kernel_initializer='he_uniform', activation='relu' , padding="same") (Added_c6)
output = tf.keras.layers.Conv2DTranspose(1, (7, 7), strides=(32, 32), activation='sigmoid', padding="same") (Added_c6)


FCN_base_model_lr_1 = tf.keras.Model(inputs=[inputImg], outputs=[output])
FCN_base_model_lr_01 = tf.keras.Model(inputs=[inputImg], outputs=[output])
FCN_base_model_lr_001 = tf.keras.Model(inputs=[inputImg], outputs=[output])


**OverFitting a Small Batch of Data**

In [ ]:
small_train_images = train_images [:100]
small_train_masks = train_masks [:100]
small_val_images = val_images [:20]
small_val_masks = val_masks [:20]


FCN_base_model_lr_1.compile(optimizer= tf.keras.optimizers.SGD(learning_rate=0.1, momentum=0.9),
    loss=[jacard_coef_loss],
    metrics=['accuracy',jacard_coef,tf.keras.metrics.MeanIoU(num_classes=2)])

FCN_base_model_lr_1.summary()






FCN_base_model_overfitting_results = FCN_base_model_lr_1.fit({'image': small_train_images}, small_train_masks, batch_size=5, epochs=50,callbacks=callbacks,
                             validation_data=({ 'image' : small_val_images}, small_val_masks))

In [ ]:
loss = FCN_base_model_overfitting_results.history['loss']
#val_loss = FCN_base_model_overfitting_results.history['val_loss']
epochs = range(len(loss))
plt.plot(epochs, loss, 'bo', label='Training loss')
#plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training loss')
plt.legend()
plt.show()

**FCN Model  with Learning rate = 0.1**

In [ ]:
'''
model.load_weights('model-v1.h5')
FCN_base_model_results = model.fit({'image': train_images}, train_masks, batch_size=32, epochs=30,callbacks=callbacks,
                             validation_data=({ 'image' : val_images}, val_masks))
                             
'''                             

In [ ]:
# Learning Rate= 0.1
FCN_base_model_lr_1.compile(optimizer= tf.keras.optimizers.SGD(learning_rate=0.1, momentum=0.9),
    loss=[jacard_coef_loss],
    metrics=['accuracy',jacard_coef,tf.keras.metrics.MeanIoU(num_classes=2)])

FCN_base_model_lr_1.summary()

In [ ]:
FCN_base_model_lr_1.load_weights('model-v1.h5')

FCN_base_model_lr_1_results = FCN_base_model_lr_1.fit({'image': train_images}, train_masks, batch_size=64, epochs=50, callbacks=callbacks,
                    validation_data=({ 'image' : val_images}, val_masks))

In [ ]:
loss = FCN_base_model_lr_1_results.history['loss']
val_loss = FCN_base_model_lr_1_results.history['val_loss']
epochs = range(len(loss))
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
FCN_base_model_lr_1_score = FCN_base_model_lr_1.evaluate(test_images, test_masks, verbose=0)
print(f'Test loss: {FCN_base_model_lr_1_score[0]} / Test accuracy: {FCN_base_model_lr_1_score[1]}')

In [ ]:

FCN_base_model_lr_1_pred = FCN_base_model_lr_1.predict(val_images)


plt.figure()
plt.title( "Predicted img")
plt.imshow(FCN_base_model_lr_1_pred[30])

plt.figure()
plt.title( "Val img")
plt.imshow(val_images[30])

plt.figure()
plt.title( "Ground truth")
plt.imshow(val_masks[30])

**FCN Model  with Learning rate = 0.01**

In [ ]:
# Learning Rate= 0.01
FCN_base_model_lr_01.compile(optimizer= tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9),
    loss=[jacard_coef_loss],
    metrics=['accuracy',jacard_coef,tf.keras.metrics.MeanIoU(num_classes=2)])

FCN_base_model_lr_01.summary()


In [ ]:
FCN_base_model_lr_01_results = FCN_base_model_lr_01.fit({'image': train_images}, train_masks, batch_size=64, epochs=50, callbacks=callbacks,
                    validation_data=({ 'image' : val_images}, val_masks))

In [ ]:
loss = FCN_base_model_lr_01_results.history['loss']
val_loss = FCN_base_model_lr_01_results.history['val_loss']
epochs = range(len(loss))
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
FCN_base_model_lr_01_score = FCN_base_model_lr_01.evaluate(test_images, test_masks, verbose=0)
print(f'Test loss: {FCN_base_model_lr_01_score[0]} / Test accuracy: {FCN_base_model_lr_01_score[1]}')

In [ ]:
FCN_base_model_lr_01_pred = FCN_base_model_lr_01.predict(val_images)


plt.figure()
plt.title( "Predicted img")
plt.imshow(FCN_base_model_lr_1_pred[10])

plt.figure()
plt.title( "Val img")
plt.imshow(val_images[10])

plt.figure()
plt.title( "Ground truth")
plt.imshow(val_masks[10])

**FCN Model  with Learning rate = 0.001**

In [ ]:
# Learning Rate= 0.001
FCN_base_model_lr_001.compile(optimizer= tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.9),
    loss=[jacard_coef_loss],
    metrics=['accuracy',jacard_coef,tf.keras.metrics.MeanIoU(num_classes=2)])


FCN_base_model_lr_001.summary()


In [ ]:
FCN_base_model_lr_001_results = FCN_base_model_lr_001.fit({'image': train_images}, train_masks, batch_size=64, epochs=50, callbacks=callbacks,
                    validation_data=({ 'image' : val_images}, val_masks))

In [ ]:
loss = FCN_base_model_lr_001_results.history['loss']
val_loss = FCN_base_model_lr_001_results.history['val_loss']
epochs = range(len(loss))
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
FCN_base_model_lr_001_score = FCN_base_model_lr_001.evaluate(test_images, test_masks, verbose=0)
print(f'Test loss: {FCN_base_model_lr_001_score[0]} / Test accuracy: {FCN_base_model_lr_001_score[1]}')

**Predicting the Results**

In [ ]:
x=FCN_base_model.predict(np.expand_dims(small_train_images[15],axis=1))

In [ ]:
plt.figure()
plt.imshow(x[0])

# **Pretrained FCN-VGG16**

In [ ]:
base_model = tf.keras.applications.VGG16(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(256, 256, 3),
    include_top=False)  # Do not include the ImageNet classifier at the top.
base_model.trainable = False



In [ ]:
inputImg = tf.keras.layers.Input((256,256,3) , name='image')
#s = Lambda(lambda x: x / 255)(inputImg)   
x = base_model(inputImg,training=False)
#tf.keras.layers.Flatten()(model.layers[-1].output)
imageModelOutput = tf.keras.layers.GlobalAveragePooling2D()(x)
#b1 = tf.keras.layers.BatchNormalization()(imageModelOutput)
#d1 = tf.keras.layers.Dense(16, activation='relu' )(imageModelOutput)
#d1 = tf.keras.layers.Dense(32, activation='relu' )(d1)
#d1 = tf.keras.layers.Dense(64, activation='relu' )(d1)
#d1 = tf.keras.layers.Dense(128, activation='relu' )(d1)
d1 = tf.keras.layers.Dense(256, activation='relu' )(imageModelOutput)

reshaped = tf.keras.layers.Reshape((16, 16,1), input_shape=(256,)) (d1)

c1Dec = tf.keras.layers.Conv2DTranspose(16, (3, 3),strides=(2, 2), activation='relu', padding="same") (reshaped)
c1Dec = tf.keras.layers.Conv2DTranspose(32, (3, 3),strides=(2, 2), activation='relu', padding="same") (c1Dec)
c1Dec = tf.keras.layers.Conv2DTranspose(64, (3, 3),strides=(2, 2), activation='relu', padding="same") (c1Dec)
c1Dec = tf.keras.layers.Conv2DTranspose(2, (3, 3),strides=(2, 2), activation='relu', padding="same") (c1Dec)
#c4Dec = tf.keras.layers.UpSampling2D(size=(16, 16))(reshaped)
finalOutput = tf.keras.layers.Conv2D(1, (1, 1),activation='sigmoid') (c1Dec)



FCN_Pretrained_lr_1 = tf.keras.Model(inputs=[inputImg], outputs=[finalOutput])
FCN_Pretrained_lr_01 = tf.keras.Model(inputs=[inputImg], outputs=[finalOutput])
FCN_Pretrained_lr_001 = tf.keras.Model(inputs=[inputImg], outputs=[finalOutput])


**PreTrained-FCN Model  with Learning rate = 0.1**

In [ ]:
FCN_Pretrained_lr_1.compile(optimizer= tf.keras.optimizers.SGD(learning_rate=0.1, momentum=0.9),
    loss=[jacard_coef_loss],
    metrics=['accuracy',jacard_coef,tf.keras.metrics.MeanIoU(num_classes=2)])

FCN_Pretrained_lr_1.summary()


In [ ]:
FCN_Pretrained_lr_1_results = FCN_Pretrained_lr_1.fit({'image': train_images}, train_masks, batch_size=64, epochs=50, callbacks=callbacks,
                    validation_data=({ 'image' : val_images}, val_masks))

In [ ]:
loss = FCN_Pretrained_lr_1_results.history['loss']
val_loss = FCN_Pretrained_lr_1_results.history['val_loss']
epochs = range(len(loss))
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
FCN_Pretrained_lr_1_score = FCN_Pretrained_lr_1.evaluate(test_images, test_masks, verbose=0)
print(f'Test loss: {FCN_Pretrained_lr_1_score[0]} / Test accuracy: {FCN_Pretrained_lr_1_score[1]}')

In [ ]:
FCN_Pretrained_lr_1_pred = FCN_Pretrained_lr_1.predict(val_images)


plt.figure()
plt.title( "Predicted img")
plt.imshow(FCN_Pretrained_lr_1_pred[10])

plt.figure()
plt.title( "Val img")
plt.imshow(val_images[10])

plt.figure()
plt.title( "Ground truth")
plt.imshow(val_masks[10])

**PreTrained-FCN Model  with Learning rate = 0.01**

In [ ]:
FCN_Pretrained_lr_01.compile(optimizer= tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9),
    loss=[jacard_coef_loss],
    metrics=['accuracy',jacard_coef,tf.keras.metrics.MeanIoU(num_classes=2)])

FCN_Pretrained_lr_01.summary()

In [ ]:
FCN_Pretrained_lr_01_results = FCN_Pretrained_lr_01.fit({'image': train_images}, train_masks, batch_size=64, epochs=50, callbacks=callbacks,
                    validation_data=({ 'image' : val_images}, val_masks))

In [ ]:
loss = FCN_Pretrained_lr_01_results.history['loss']
val_loss = FCN_Pretrained_lr_01_results.history['val_loss']
epochs = range(len(loss))
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
FCN_Pretrained_lr_01_score = FCN_Pretrained_lr_01.evaluate(test_images, test_masks, verbose=0)
print(f'Test loss: {FCN_Pretrained_lr_01_score[0]} / Test accuracy: {FCN_Pretrained_lr_01_score[1]}')

In [ ]:
FCN_Pretrained_lr_01_pred = FCN_Pretrained_lr_01.predict(val_images)


plt.figure()
plt.title( "Predicted img")
plt.imshow(FCN_Pretrained_lr_01_pred[10])

plt.figure()
plt.title( "Val img")
plt.imshow(val_images[10])

plt.figure()
plt.title( "Ground truth")
plt.imshow(val_masks[10])

**PreTrained-FCN Model  with Learning rate = 0.001**

In [ ]:
FCN_Pretrained_lr_001.compile(optimizer= tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.9),
    loss=[jacard_coef_loss],
    metrics=['accuracy',jacard_coef,tf.keras.metrics.MeanIoU(num_classes=2)])

FCN_Pretrained_lr_001.summary()

In [ ]:
FCN_Pretrained_lr_001_results = FCN_Pretrained_lr_001.fit({'image': train_images}, train_masks, batch_size=64, epochs=50, callbacks=callbacks,
                    validation_data=({ 'image' : val_images}, val_masks))

In [ ]:
loss = FCN_Pretrained_lr_001_results.history['loss']
val_loss = FCN_Pretrained_lr_001_results.history['val_loss']
epochs = range(len(loss))
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
FCN_Pretrained_lr_001_score = FCN_Pretrained_lr_001.evaluate(test_images, test_masks, verbose=0)
print(f'Test loss: {FCN_Pretrained_lr_001_score[0]} / Test accuracy: {FCN_Pretrained_lr_001_score[1]}')

In [ ]:
FCN_Pretrained_lr_001_pred = FCN_Pretrained_lr_001.predict(val_images)


plt.figure()
plt.title( "Predicted img")
plt.imshow(FCN_Pretrained_lr_001_pred[10])

plt.figure()
plt.title( "Val img")
plt.imshow(val_images[10])

plt.figure()
plt.title( "Ground truth")
plt.imshow(val_masks[10])

# U-NET

In [ ]:
inputImg = tf.keras.layers.Input((256,256,3) , name='image')
s = Lambda(lambda x: x / 255)(inputImg)   
c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(s)
#c1 = Dropout(0.1)(c1)
c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
p1 = MaxPooling2D((2, 2))(c1)

c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
#c2 = Dropout(0.1)(c2)
c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
p2 = MaxPooling2D((2, 2))(c2)

c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
#c3 = Dropout(0.2)(c3)
c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
p3 = MaxPooling2D((2, 2))(c3)

c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
#c4 = Dropout(0.2)(c4)
c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
p4 = MaxPooling2D(pool_size=(2, 2))(c4)

c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
#c5 = Dropout(0.3)(c5)
c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)

#Expansive path 
u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
u6 = concatenate([u6, c4])
c6 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
#c6 = Dropout(0.2)(c6)
c6 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)

u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
u7 = concatenate([u7, c3])
c7 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
#c7 = Dropout(0.2)(c7)
c7 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)

u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
u8 = concatenate([u8, c2])
c8 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
#c8 = Dropout(0.1)(c8)
c8 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)

u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
u9 = concatenate([u9, c1], axis=3)
c9 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
#c9 = Dropout(0.1)(c9)
c9 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)

finalOutput = Conv2D(1, (1, 1), activation='sigmoid')(c9)




UNET_lr_1 = tf.keras.Model(inputs=[inputImg], outputs=[finalOutput])
UNET_lr_01 = tf.keras.Model(inputs=[inputImg], outputs=[finalOutput])
UNET_lr_001 = tf.keras.Model(inputs=[inputImg], outputs=[finalOutput])

In [ ]:
'''

model.compile(optimizer=tf.keras.optimizers.RMSprop(
    learning_rate=0.0001, rho=0.9, momentum=0.0, epsilon=1e-07, centered=False,
    name='RMSprop'),
    loss=[jacard_coef_loss],
    metrics=[[jacard_coef],'accuracy'])
#model.compile(optimizer = 'adam', loss = [jacard_coef_loss], metrics = [jacard_coef])

model.summary()
'''

**U-Net Model  with Learning rate = 0.1**

In [ ]:
UNET_lr_1.compile(optimizer=tf.keras.optimizers.RMSprop(
    learning_rate=0.0001, rho=0.9, momentum=0.0, epsilon=1e-07, centered=False,
    name='RMSprop'),
    loss=[jacard_coef_loss],
    metrics=[[jacard_coef],'accuracy'])
#model.compile(optimizer = 'adam', loss = [jacard_coef_loss], metrics = [jacard_coef])

UNET_lr_1.summary()

In [ ]:
UNET_lr_1_results = UNET_lr_1.fit({'image': train_images}, train_masks, batch_size=64, epochs=50, callbacks=callbacks,
                    validation_data=({ 'image' : val_images}, val_masks))

In [ ]:
loss = UNET_lr_1_results.history['loss']
val_loss = UNET_lr_1_results.history['val_loss']
epochs = range(len(loss))
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
UNET_lr_1_score = UNET_lr_1.evaluate(test_images, test_masks, verbose=0)
print(f'Test loss: {UNET_lr_1_score[0]} / Test accuracy: {UNET_lr_1_score[1]}')

In [ ]:
UNET_lr_1_pred = UNET_lr_1.predict(test_images)


plt.figure()
plt.title( "Predicted img")
plt.imshow(UNET_lr_1_pred[50])

plt.figure()
plt.title( "Val img")
plt.imshow(test_images[50])

plt.figure()
plt.title( "Ground truth")
plt.imshow(test_masks[50])

**U-Net Model  with Learning rate = 0.01**

In [ ]:
UNET_lr_01.compile(optimizer=tf.keras.optimizers.RMSprop(
    learning_rate=0.0001, rho=0.9, momentum=0.0, epsilon=1e-07, centered=False,
    name='RMSprop'),
    loss=[jacard_coef_loss],
    metrics=[[jacard_coef],'accuracy'])
#model.compile(optimizer = 'adam', loss = [jacard_coef_loss], metrics = [jacard_coef])

UNET_lr_01.summary()

In [ ]:
UNET_lr_01_results = UNET_lr_01.fit({'image': train_images}, train_masks, batch_size=64, epochs=50, callbacks=callbacks,
                    validation_data=({ 'image' : val_images}, val_masks))

In [ ]:
loss = UNET_lr_01_results.history['loss']
val_loss = UNET_lr_01_results.history['val_loss']
epochs = range(len(loss))
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
UNET_lr_01_score = UNET_lr_01.evaluate(test_images, test_masks, verbose=0)
print(f'Test loss: {UNET_lr_01_score[0]} / Test accuracy: {UNET_lr_01_score[1]}')

In [ ]:
UNET_lr_01_pred = UNET_lr_01.predict(test_images)


plt.figure()
plt.title( "Predicted img")
plt.imshow(UNET_lr_1_pred[50])

plt.figure()
plt.title( "Val img")
plt.imshow(test_images[50])

plt.figure()
plt.title( "Ground truth")
plt.imshow(test_masks[50])

**U-Net Model  with Learning rate = 0.001**

In [ ]:
UNET_lr_001.compile(optimizer=tf.keras.optimizers.RMSprop(
    learning_rate=0.0001, rho=0.9, momentum=0.0, epsilon=1e-07, centered=False,
    name='RMSprop'),
    loss=[jacard_coef_loss],
    metrics=[[jacard_coef],'accuracy'])
#model.compile(optimizer = 'adam', loss = [jacard_coef_loss], metrics = [jacard_coef])

UNET_lr_001.summary()

In [ ]:
UNET_lr_001_results = UNET_lr_001.fit({'image': train_images}, train_masks, batch_size=64, epochs=50, callbacks=callbacks,
                    validation_data=({ 'image' : val_images}, val_masks))

In [ ]:
loss = UNET_lr_001_results.history['loss']
val_loss = UNET_lr_001_results.history['val_loss']
epochs = range(len(loss))
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
UNET_lr_001_score = UNET_lr_001.evaluate(test_images, test_masks, verbose=0)
print(f'Test loss: {UNET_lr_001_score[0]} / Test accuracy: {UNET_lr_001_score[1]}')

In [ ]:
UNET_lr_001_pred = UNET_lr_001.predict(test_images)


plt.figure()
plt.title( "Predicted img")
plt.imshow(UNET_lr_1_pred[50])

plt.figure()
plt.title( "Val img")
plt.imshow(test_images[50])

plt.figure()
plt.title( "Ground truth")
plt.imshow(test_masks[50])

In [ ]:
FCN_base_model_results = FCN_base_model.fit({'image': small_train_images}, small_train_masks, batch_size=5, epochs=50,callbacks=callbacks,
                             validation_data=({ 'image' : small_val_images}, small_val_masks))